# Covid 19 Vaccine Progress Indicators Analysis

### Import Dependencies

In [1]:
#Data Cleaning
import pandas as pd
import numpy as np
from scipy.stats import linregress

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sn

# Data Extraction
import requests
import json
import time

## Import CSV file

In [2]:
#Covid vaccinations per country
df = pd.read_csv('./Resources/country_vaccinations.csv')

### CSV Data Cleaning

In [3]:
# Group by country iso code and find the most recent count of people vaccinated / percentage of people vaccinated
countryVacc_df = df.groupby('iso_code').agg({'total_vaccinations':'max','people_fully_vaccinated_per_hundred':'max'}).dropna()
countryVacc_df = countryVacc_df[(countryVacc_df['people_fully_vaccinated_per_hundred'] != 0)]

In [4]:
# Reset index to include iso code as value
vaccination_df= countryVacc_df.reset_index()

In [5]:
#Save to CSV file for later access
vaccination_df.to_csv('./Resources/iso_code.csv', index=False)

## Import World Bank JSON data with API

In [6]:
#read csv of iso codes
file = './Resources/iso_code.csv'
iso_code_df = pd.read_csv(file)

In [7]:
#adding empty columns for indicator values to add to dataframe
iso_code_df['GDP'] = ''
iso_code_df['Total_Pop'] = ''
iso_code_df['Pop_Den'] = ''
iso_code_df['health_exp'] = ''

In [8]:
# JSON requests from world bank database

# Indicators and date we want to request
variable = ['NY.GDP.MKTP.CD','SP.POP.TOTL.','EN.POP.DNST','SH.XPD.PVTD.CH.ZS']
date = '2018'


for j, var in enumerate(variable): # loop through all indicators requested
    for i, iso in enumerate(iso_code_df['iso_code']): # Loop through all iso codes in dataframe
        try:
            #keep track of how long the request takes
            tic = time.perf_counter()
            
            #base url for making indicator requests
            base_url = f'http://api.worldbank.org/v2/country/{iso}/indicator/{var}?date={date}&format=json'
            
            #request the modified url and recast as json
            response = requests.get(base_url).json()
            
            #index json object
            responseJSON = response[1][0]['value']
            
            #add response to dataframe
            iso_code_df.iloc[i,3+j] = responseJSON
            
            #stop time
            toc = time.perf_counter()
            
            #print statement to monitor progress
            print(f'{iso},{var}: Extraction finished in {toc - tic:0.4f}s.',end='\r')

        except: 
            print(f'Error {iso},{var} data missing, time:{toc - tic:0.4f}s.')
 

Error AIA,NY.GDP.MKTP.CD data missing.d in 0.3933s.
Error FLK,NY.GDP.MKTP.CD data missing.d in 0.2120s.
Error GGY,NY.GDP.MKTP.CD data missing.d in 0.1810s.
Error JEY,NY.GDP.MKTP.CD data missing.d in 0.2054s.
Error MSR,NY.GDP.MKTP.CD data missing.d in 0.2022s.
Error OWID_CYN,NY.GDP.MKTP.CD data missing.0.1690s.
Error OWID_ENG,NY.GDP.MKTP.CD data missing.
Error OWID_NIR,NY.GDP.MKTP.CD data missing.
Error OWID_SCT,NY.GDP.MKTP.CD data missing.
Error OWID_WLS,NY.GDP.MKTP.CD data missing.
Error SHN,NY.GDP.MKTP.CD data missing.d in 0.3230s.
Error AIA,SP.POP.TOTL. data missing.d in 0.2276s.s.
Error FLK,SP.POP.TOTL. data missing.d in 0.1921s.
Error GGY,SP.POP.TOTL. data missing.d in 0.1572s.
Error JEY,SP.POP.TOTL. data missing.d in 0.1874s.
Error MSR,SP.POP.TOTL. data missing.d in 0.1469s.
Error OWID_CYN,SP.POP.TOTL. data missing.0.1690s.
Error OWID_ENG,SP.POP.TOTL. data missing.
Error OWID_NIR,SP.POP.TOTL. data missing.
Error OWID_SCT,SP.POP.TOTL. data missing.
Error OWID_WLS,SP.POP.TOTL. data

### Data Cleaning

In [9]:
# Drop rows with missing values
iso_code_df.replace('',np.NaN,inplace=True)
iso_cleaned_df = iso_code_df.dropna()

In [10]:
iso_cleaned_df.head(10)

,iso_code,total_vaccinations,people_fully_vaccinated_per_hundred,GDP,Total_Pop,Pop_Den,health_exp
2,ALB,400064.0,0.02,1.514702e+10,2866376.0,104.612263,44.657787
3,AND,26414.0,6.06,3.218316e+09,77006.0,163.842553,32.065220
4,ARE,10336214.0,38.79,4.222150e+11,9630959.0,135.609110,48.421539
5,ARG,7386650.0,1.95,5.176267e+11,44494502.0,16.258510,38.290127
6,AUT,2863389.0,8.73,4.550949e+11,8840521.0,107.131859,26.914829
7,AZE,1440243.0,4.87,4.711294e+10,9939771.0,120.257592,72.804810
8,BEL,3563371.0,6.60,5.437344e+11,11427054.0,377.379590,24.215460
9,BGD,7954176.0,1.31,2.740391e+11,161356039.0,1239.579312,76.501839
10,BGR,750899.0,2.56,6.623016e+10,7025037.0,64.711100,42.363476
11,BHR,1206763.0,31.14,3.765250e+10,1569439.0,2012.101282,41.167648


### Save to CSV for future use

In [11]:
#Save df to csv
iso_cleaned_df.to_csv('./Resources/isoDataFinal.csv',index=False)